In [1]:
import pandas as pd
import creds
from sqlalchemy import create_engine, text as sa_text
from scipy import stats
import requests

In [2]:
# Construct the connection string using your credentials
con_string = f"postgresql+psycopg2://{creds.DB_USER}:{creds.DB_PASS}@{creds.DB_HOST}/{creds.DB_NAME}"

# Create a database engine and connect to the database
engine = create_engine(con_string)
conn = engine.connect()

xg_df = pd.read_sql_query(sa_text('SELECT * FROM expect_goals'), con=conn)
pbp_df = pd.read_sql_query(sa_text('SELECT * FROM play_by_play'), con=conn)

conn.commit()
conn.close()

In [3]:
pbp_xg_df = pd.merge(left=pbp_df, right=xg_df, on=['gameId','playId'], how='inner')

In [4]:
engine = create_engine(con_string)
conn = engine.connect()

player_stats_db = pd.read_sql_query(sa_text('SELECT * FROM skater_stats'), con=conn)


conn.commit()
conn.close()

In [43]:
gf_dict = {}
home_cols = [f'home{i}' for i in range(1, 7)]
away_cols = [f'away{i}' for i in range(1, 7)]
player_cols = home_cols + away_cols + ["eventPlayer1"]
player_ids = player_stats_db.id.unique()
pbp_xg_df["eventPlayer1"] = pbp_xg_df["eventPlayer1"].astype('object')

for player_id in player_ids:
    player_scope = pbp_xg_df[pbp_xg_df[player_cols].eq(str(player_id)).any(axis=1)]
    seasons = player_scope['season'].unique()
    for season in seasons:
        xg_dict = {}
        season_scope = player_scope[player_scope['season'] == season]
        for strength in ['EV', 'PP', 'SH']:
            strength_scope = season_scope[season_scope['strength'] == strength]
            xg_dict[f'{strength}_ixG'] = strength_scope[strength_scope['eventPlayer1'].eq(player_id)]['xG'].sum()
            home_on_ice_filter = (strength_scope[home_cols].eq(str(player_id)).any(axis=1))
            away_on_ice_filter = (strength_scope[away_cols].eq(str(player_id)).any(axis=1))
            
            xg_dict[f'{strength}_xGF'] = strength_scope.loc[(home_on_ice_filter & (season_scope['homeTeam'] == season_scope['eventTeam'])) | (away_on_ice_filter & (season_scope['awayTeam'] == season_scope['eventTeam'])), 'xG'].sum()
            xg_dict[f'{strength}_xGA'] = strength_scope.loc[(home_on_ice_filter & (season_scope['homeTeam'] != season_scope['eventTeam'])) | (away_on_ice_filter & (season_scope['awayTeam'] != season_scope['eventTeam'])), 'xG'].sum()

        gf_dict[(player_id, season)] = xg_dict




In [81]:
df = pd.DataFrame(gf_dict).transpose().reset_index().rename({'level_0':'id','level_1':'season'}, axis='columns')

In [82]:
player_stats_db.columns

Index(['id', 'season', 'games', 'goals', 'assists', 'points', 'shots',
       'shotPct', 'pim', 'timeOnIce', 'evenTimeOnIce', 'powerPlayTimeOnIce',
       'shortHandedTimeOnIce', 'timeOnIcePerGame', 'evenTimeOnIcePerGame',
       'powerPlayTimeOnIcePerGame', 'shortHandedTimeOnIcePerGame'],
      dtype='object')

In [83]:
player_stats_db['id'] = player_stats_db['id'].astype('string')
player_stats_db['season'] = player_stats_db['season'].astype('string')
df['id'] = df['id'].astype('string')
df['season'] = df['season'].astype('string')

toi_df = player_stats_db[['id', 'season', 'timeOnIce','evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'goals','shotPct']]
toi_xg_df = df.merge(toi_df, on=['id','season'])

In [84]:
toi_xg_df['ALL_ixG'] = toi_xg_df['EV_ixG'] + toi_xg_df['PP_ixG'] + toi_xg_df['SH_ixG']
toi_xg_df['ALL_xGF'] = toi_xg_df['EV_xGF'] + toi_xg_df['PP_xGF'] + toi_xg_df['SH_xGF']
toi_xg_df['ALL_xGA'] = toi_xg_df['EV_xGA'] + toi_xg_df['PP_xGA'] + toi_xg_df['SH_xGA']
toi_xg_df['goals_above_expected'] = toi_xg_df['goals'] - toi_xg_df['ALL_ixG']

In [87]:
def time_to_minutes(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        total_minutes = minutes + seconds / 60
        return total_minutes
    except ValueError:
        print("Invalid time format. Please use 'MM:SS' format.")
        return None

toi_xg_df[['timeOnIceMin', 'evenTimeOnIceMin','powerPlayTimeOnIceMin','shortHandedTimeOnIceMin']] = toi_xg_df[['timeOnIce','evenTimeOnIce','powerPlayTimeOnIce','shortHandedTimeOnIce']].applymap(lambda x: round(time_to_minutes(x), 2))

In [178]:
conn = engine.connect()
toi_xg_df.to_sql('advanced_stats_raw',if_exists='replace', con=conn)
conn.close()

### Skater Stat List:
Deployment
- TOI all types x EV, PP, SH

Offence
 - ixG x
 - xGF x
 - EV xGF x
 - PP XGF x
 - Goals  x
 - Goals - ixG x
 
Defence
 - xGA x 
 - EV xGA x
 - SH XGA x
 - EV GA
 
Misc
 - Penalties Taken x
 - Penalties Drawn x
 - Faceoff differential x
 


 
 
 


In [151]:
conn = engine.connect()
toi_xg_df = pd.read_sql('advanced_stats_raw', con=conn)
conn.close()

In [152]:
faceoff_url = f'https://api.nhle.com/stats/rest/en/skater/faceoffwins?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22totalFaceoffWins%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start=0&limit=100&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
faceoff_dict = {}
resp = requests.get(faceoff_url)
resp.raise_for_status()
json_data = resp.json()
total_events = json_data['total']

for start_index in range(0,total_events+1,50):
    faceoff_url = f'https://api.nhle.com/stats/rest/en/skater/faceoffwins?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22totalFaceoffWins%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22faceoffWinPct%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start={start_index}&limit=100&factCayenneExp=gamesPlayed%3E=0&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
    try:
        resp = requests.get(faceoff_url)
        resp.raise_for_status()
        json_data = resp.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    for player in json_data['data']:
        player_id = str(player['playerId'])
        season = str(player['seasonId'])
        faceoffDifferential  = player['totalFaceoffWins'] - player['totalFaceoffLosses']
        faceoffTotal = player['totalFaceoffs']
        faceoff_dict[(player_id, season)] = [faceoffDifferential,faceoffTotal]

toi_xg_df['faceoff_differential'] = toi_xg_df.apply(lambda row: faceoff_dict[(row['id'], row['season'])][0], axis=1)
toi_xg_df['faceoff_total'] = toi_xg_df.apply(lambda row: faceoff_dict[(row['id'], row['season'])][1], axis=1)

In [153]:
penalties_url = f'https://api.nhle.com/stats/rest/en/skater/penalties?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22lastName%22,%22direction%22:%22ASC_CI%22%7D,%7B%22property%22:%22skaterFullName%22,%22direction%22:%22ASC_CI%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start=0&limit=100&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
penalties_dict = {}
resp = requests.get(penalties_url)
resp.raise_for_status()
json_data = resp.json()
total_events = json_data['total']

for start_index in range(0,total_events+1,50):
    penalties_url = f'https://api.nhle.com/stats/rest/en/skater/penalties?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22lastName%22,%22direction%22:%22ASC_CI%22%7D,%7B%22property%22:%22skaterFullName%22,%22direction%22:%22ASC_CI%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start={start_index}&limit=100&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
    try:
        resp = requests.get(penalties_url)
        resp.raise_for_status()
        json_data = resp.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    for player in json_data['data']:
        player_id = str(player['playerId'])
        season = str(player['seasonId'])
        penaliesTaken  = player['penalties']
        penaliesDrawn = player['penaltiesDrawn']
        penalties_dict[(player_id, season)] = [penaliesTaken,penaliesDrawn]

toi_xg_df['penalties_taken'] = toi_xg_df.apply(lambda row: penalties_dict[(row['id'], row['season'])][0], axis=1)
toi_xg_df['penalties_drawn'] = toi_xg_df.apply(lambda row: penalties_dict[(row['id'], row['season'])][1], axis=1)

In [154]:
goals_against_url = f'https://api.nhle.com/stats/rest/en/skater/goalsForAgainst?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22seasonId%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start=0&limit=100&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
goals_against_dict = {}
resp = requests.get(penalties_url)
resp.raise_for_status()
json_data = resp.json()
total_events = json_data['total']

for start_index in range(0,total_events+1,50):
    goals_against_url = f'https://api.nhle.com/stats/rest/en/skater/goalsForAgainst?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22seasonId%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start={start_index}&limit=100&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20222023%20and%20seasonId%3E=20182019'
    try:
        resp = requests.get(goals_against_url)
        resp.raise_for_status()
        json_data = resp.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    for player in json_data['data']:
        player_id = str(player['playerId'])
        season = str(player['seasonId'])
        evenStrengthGoalsAgainst  = player['evenStrengthGoalsAgainst']
        goals_against_dict[(player_id, season)] = [evenStrengthGoalsAgainst]

toi_xg_df['ev_goals_against'] = toi_xg_df.apply(lambda row: goals_against_dict[(row['id'], row['season'])][0], axis=1)


In [157]:
# Define column prefixes and corresponding time columns
columns = [
    ('EV', 'evenTimeOnIceMin'),
    ('PP', 'powerPlayTimeOnIceMin'),
    ('SH', 'shortHandedTimeOnIceMin'),
    ('ALL', 'timeOnIceMin')
]
# Iterate through column prefixes and time columns to calculate rates and percentiles
for prefix, time_column in columns:
    toi_xg_df[f'{prefix}_ixG_rates'] = toi_xg_df.apply(lambda row: row[f'{prefix}_ixG'] / row[time_column] if row[time_column] != 0 else 0.0, axis=1)
    toi_xg_df[f'{prefix}_xGF_rates'] = toi_xg_df.apply(lambda row: row[f'{prefix}_xGF'] / row[time_column] if row[time_column] != 0 else 0.0, axis=1)
    toi_xg_df[f'{prefix}_xGA_rates'] = toi_xg_df.apply(lambda row: row[f'{prefix}_xGA'] / row[time_column] if row[time_column] != 0 else 0.0, axis=1)

toi_xg_df[f'penalties_taken_rates'] = toi_xg_df.apply(lambda row: row[f'penalties_taken'] / row['timeOnIceMin'] if row['timeOnIceMin'] != 0 else 0.0, axis=1)
toi_xg_df[f'penalties_drawn_rates'] = toi_xg_df.apply(lambda row: row[f'penalties_drawn'] / row['timeOnIceMin'] if row['timeOnIceMin'] != 0 else 0.0, axis=1)
toi_xg_df[f'goal_rates'] = toi_xg_df.apply(lambda row: row[f'goals'] / row['timeOnIceMin'] if row['timeOnIceMin'] != 0 else 0.0, axis=1)

toi_xg_df[f'ev_goals_against_rate'] = toi_xg_df.apply(lambda row: row[f'ev_goals_against'] / row['evenTimeOnIceMin'] if row['evenTimeOnIceMin'] != 0 else 0.0, axis=1)

# Calculate season-wise percentiles
toi_xg_df['faceoff_differential_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['faceoff_total'] > 50) & (toi_xg_df['season'] == row['season']), 'faceoff_differential'], row['faceoff_differential'], kind='rank'), axis=1)
toi_xg_df['goals_above_expected_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), 'goals_above_expected'], row['goals_above_expected'], kind='rank'), axis=1)
# toi_xg_df['pim_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(-toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 400) & (toi_xg_df['season'] == row['season']), 'pim'], -row['pim'], kind='rank'), axis=1)
toi_xg_df['penalties_taken_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(-toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 500) & (toi_xg_df['season'] == row['season']), 'penalties_taken_rates'], -row['penalties_taken_rates'], kind='rank'), axis=1)
toi_xg_df['penalties_drawn_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), 'penalties_drawn_rates'], row['penalties_drawn_rates'], kind='rank'), axis=1)
toi_xg_df['goal_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), 'goal_rates'], row['goal_rates'], kind='rank'), axis=1)

toi_xg_df['ev_goals_against_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(-toi_xg_df.loc[(toi_xg_df['evenTimeOnIceMin'] > 400) & (toi_xg_df['season'] == row['season']), 'ev_goals_against_rate'], -row['ev_goals_against_rate'], kind='rank'), axis=1)



for prefix, time_column in columns:
        toi_xg_df[f'{prefix}_TOI_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), time_column], row[time_column], kind='rank'), axis=1)
        toi_xg_df[f'{prefix}_ixG_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), f'{prefix}_ixG_rates'], row[f'{prefix}_ixG_rates'], kind='rank'), axis=1)
        toi_xg_df[f'{prefix}_xGF_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), f'{prefix}_xGF_rates'], row[f'{prefix}_xGF_rates'], kind='rank'), axis=1)
        toi_xg_df[f'{prefix}_xGA_percentile'] = toi_xg_df.apply(lambda row: stats.percentileofscore(-toi_xg_df.loc[(toi_xg_df['timeOnIceMin'] > 240) & (toi_xg_df['season'] == row['season']), f'{prefix}_xGA_rates'], -row[f'{prefix}_xGA_rates'], kind='rank'), axis=1)


In [165]:
conn = engine.connect()
toi_xg_df.to_sql('percentiles',if_exists='replace', con=conn, index=False)
conn.close()

# Goalie Percentiles

### List
- High Danger Sv%
- Med Danger Sv%
- Low Danger Sv%
- SH Sv%
- EV Sv%
- GAA
- TOI
- xG Against 
- GSAx
- Rebounds